In [11]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd

from glob import glob
import json
import os
import sys
sys.path.append(os.path.abspath(os.getcwd() + "./../"))

In [12]:
import torch
from IPython.display import Image as IPythonImage
from PIL import Image, ImageDraw, ImageFont
import io
from tasks.copytask import dataloader
from train import evaluate

In [13]:
ls


Untitled.ipynb                fonts/
copy-task-plots.ipynb*        repeat-copy-task-plots.ipynb*


In [5]:
ls

Untitled.ipynb                fonts/
copy-task-plots.ipynb*        repeat-copy-task-plots.ipynb*


In [6]:
io

<module 'io' from '/anaconda3/lib/python3.7/io.py'>